In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


# 1) Data Cleaning

## configuring training and test data 

In [2]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
y = train_data['Premium Amount']
train_data = train_data.drop(['id', 'Premium Amount'], axis=1)

test_data = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')
test_ids = test_data['id']
test_data = test_data.drop(['id'], axis=1)

# function to check for NaN values in a specified column
def checknan(colname):
    return train_data[train_data[colname].isna()]

# function to fillna with either mean, median, 0, or another column
def fillNaN(colnamelist, type, othercol=None, datasetlist=[train_data, test_data]):
    if type == 'mean':
        for dataset in datasetlist:
            for colname in colnamelist:
                dataset.fillna({colname: train_data[colname].mean()}, inplace=True) # Version14: adding test_data to the fillNaN function
    elif type == 'median':
        for dataset in datasetlist:
            for colname in colnamelist:
                dataset.fillna({colname: train_data[colname].median()}, inplace=True)
    elif type == 'mode':
        for dataset in datasetlist:
            for colname in colnamelist:
                dataset.fillna({colname: train_data[colname].mode()[0]}, inplace=True)
    elif type == 0:
        for dataset in datasetlist:
            for colname in colnamelist:
                dataset.fillna({colname: 0}, inplace=True)
    elif type == 'othercol':
        for dataset in datasetlist:
            for colname in colnamelist:
                dataset.fillna({colname: train_data[othercol]}, inplace=True)
    print(f'fillNaN [{type}] complete')

# nan_cols = train_data.columns[train_data.isna().any()]
# for col in nan_cols:
#     print(f'- {col}')

# fillna for column: Age
fillNaN(['Age'], 'mean')
# fillna for column: Annual Income
fillNaN(['Annual Income'], 'median')
# fillna for column: Marital Status
fillNaN(['Marital Status'], 'mode')
# fillna for column: Number of Dependents
fillNaN(['Number of Dependents'], 'mean')
# fillna for column: Occupation
fillNaN(['Occupation'], 'mode')
# fillna for column: Health Score
fillNaN(['Health Score'], 'mean')
# fillna for column: Previous Claims
fillNaN(['Previous Claims'], 'mode')
# fillna for column: Vehicle Age
fillNaN(['Vehicle Age'], 'median')
# fillna for column: Credit Score
fillNaN(['Credit Score'], 'median')
# fillna for column: Insurance Duration
fillNaN(['Insurance Duration'], 'median')
# fillna for column: Customer Feedback
fillNaN(['Customer Feedback'], 'mode')

# processing the values in the 'Policy Start Date', 'Gender', 'Customer Feedback', and 'Smoking Status' columns
# starting with 'Policy Start Date'
train_data['Policy Start Date'] = pd.to_datetime(train_data['Policy Start Date'])
train_data['start year'] = train_data['Policy Start Date'].dt.year
train_data['start month'] = train_data['Policy Start Date'].dt.month
train_data = train_data.drop(['Policy Start Date'], axis=1)
print(train_data.head())
# on to 'Gender'
gendermap = {'Male': 1, 'Female': 0}
train_data['Gender'] = train_data['Gender'].map(gendermap)
# Customer Feedback
feedbackmap = {'Poor': 1, 'Average': 2, 'Good': 3}
train_data['Customer Feedback'] = train_data['Customer Feedback'].map(feedbackmap)
# Smoking Status
smokingmap = {'Yes': 1, 'No': 0}
train_data['Smoking Status'] = train_data['Smoking Status'].map(smokingmap)




fillNaN [mean] complete
fillNaN [median] complete
fillNaN [mode] complete
fillNaN [mean] complete
fillNaN [mode] complete
fillNaN [mean] complete
fillNaN [mode] complete
fillNaN [median] complete
fillNaN [median] complete
fillNaN [median] complete
fillNaN [mode] complete
    Age  Gender  Annual Income Marital Status  Number of Dependents  \
0  19.0  Female        10049.0        Married                   1.0   
1  39.0  Female        31678.0       Divorced                   3.0   
2  23.0    Male        25602.0       Divorced                   3.0   
3  21.0    Male       141855.0        Married                   2.0   
4  21.0    Male        39651.0         Single                   1.0   

  Education Level     Occupation  Health Score  Location    Policy Type  \
0      Bachelor's  Self-Employed     22.598761     Urban        Premium   
1        Master's       Employed     15.569731     Rural  Comprehensive   
2     High School  Self-Employed     47.177549  Suburban        Premium   
3

# 2) Model Configuration

## RandomForestRegressor

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rfrmodel = RandomForestRegressor